### Regularized Tree Ensemble Code

In [ ]:
# Load Required Packages
import sklearn
import statistics
import scipy as sc
import numpy as np
import pandas as pd
import random
import cvxpy as cp
import matplotlib.pyplot as plt
import multiprocessing as mp
from multiprocessing import Pool
import time
import re
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import mosek
import statsmodels.api as sm

def converge_test(sequence, threshold,length):
    diff = np.diff(sequence)
    if len(diff) < (length+1):
        return False
    else:
        return ( max(np.abs(diff[-length:])) < threshold)
    
def compute_loss(y, y_hat): 
    return ((y - y_hat) ** 2) / 2

def loss_gradient(y, y_hat): 
    return -(y-y_hat) 

def plot_tradeoff_curve(test_acc,nonzero,color,label):
    results = pd.DataFrame()
    for i in range(0,len(test_acc)):
        results = results.append(pd.DataFrame(np.column_stack((test_acc[i],nonzero[i])),columns = ['test_acc','nonzero']))
    agg = results.groupby(['nonzero'], as_index=False).agg({'test_acc':['mean','std','count']})
    
    plt.plot(agg['nonzero'],agg['test_acc']['mean'],color = color,label = label)
    plt.errorbar(agg['nonzero'],agg['test_acc']['mean'], agg['test_acc']['std'],color = color)
    plt.xlabel('Number of Nonzero Features')
    plt.ylabel('Test Error')
    plt.legend()

def build_trees_bag(arg):
    xTrain = arg[0]
    yTrain = arg[1]
    xTest= arg[2]
    yTest= arg[3]
    max_depth= arg[4]
    lambd= arg[5]
    threshold= arg[6] 
    sketch = arg[7]
    problem_type = arg[8]
    #not used
    learning_rate = arg[9]
    n_estimators = arg[10]
    
    train = xTrain
    train = train.reset_index().drop('index',axis = 1)
    train['yTrain'] = list(yTrain)

    features = xTrain.columns
    nfeatures = len(features)
    importance_key = pd.DataFrame(features,columns = ['Features'])
    
    tree_results = []
    i = 0
    depth = 1
    total_trees = 0
    
    for depth in range (1,max_depth+1):
        i = 0
        ### Early Stopping
        early_stop_pred = []
        early_stop_train_err = []
        converged = False
        
        while converged == False:
            train1 = train.sample(n = len(train), replace = True)
        
            yTrain1 = train1['yTrain']
            xTrain1 = train1[features]
            
            if problem_type == 'Regression':
                clf = DecisionTreeRegressor(max_depth = depth)
            elif problem_type == 'Classification':
                clf = DecisionTreeClassifier(max_depth = depth)
        
            clf.fit(xTrain1,yTrain1)
            
            imp = pd.DataFrame(np.column_stack((xTrain1.columns,clf.feature_importances_)), columns = ['Features','Importances'])
            used = imp[imp['Importances']>0]['Features'].values
            feature_indicator = [int(x in used) for x in features]
            pred = clf.predict(xTrain[features])
            feature_importances = pd.merge(importance_key,imp, on = 'Features', how = 'left').fillna(0)['Importances'].values
            test_pred = clf.predict(xTest[features])
            tree_results.append([pred,feature_indicator,feature_importances, test_pred  ,clf,xTrain1,yTrain1,features])
            i = i+1
            total_trees = total_trees+1
            early_stop_pred.append(pred)
            early_stop_train_err.append(np.sqrt(np.mean((np.mean(early_stop_pred,axis = 0) - yTrain)**2)))
            converged = converge_test(early_stop_train_err,10**-2,3)
    
    return tree_results

def solve_step_nonsketch(arg, tree_results):
    
    xTrain = arg[0]
    yTrain = arg[1]
    xTest= arg[2]
    yTest= arg[3]
    max_depth= arg[4]
    lambd= arg[5]
    threshold= arg[6] 
    sketch = arg[7]
    problem_type = arg[8]
    learning_rate = arg[9]
    n_estimators = arg[10]
    feature_list = xTrain.columns
    
    tree_pred = np.transpose(np.array([np.array(row[0]) for row in tree_results]))
    test_pred = np.transpose(np.array([np.array(row[3]) for row in tree_results]))
    indicators = np.transpose(np.array([np.array(row[1]) for row in tree_results]))
    w = cp.Variable(len(tree_results),nonneg=True)
    
    if problem_type == 'Regression':
        
        objective = 0.5 * (1/len(yTrain))*cp.sum_squares(cp.matmul(tree_pred,w)-yTrain) + lambd*cp.norm(cp.matmul(indicators,w),1)
        prob = cp.Problem(cp.Minimize(objective) )
        prob.solve(solver = cp.MOSEK,mosek_params = {mosek.dparam.optimizer_max_time: 100.0} )
        weights = np.asarray(w.value)
        low_values_flags = np.abs(weights) < threshold  
        weights[low_values_flags] = 0 

        tree_ind = np.where(weights >0)[0]
        if len(tree_ind)==0:
            train_error =  np.sqrt(np.mean((yTrain )**2))
            test_error = np.sqrt(np.mean((yTest )**2))
            return([[],test_error,0,train_error])

        importances = np.array([np.array(row[2]) for row in tree_results])
        feature_importances = np.mean(importances[tree_ind],axis = 0)
        nonzero_features = xTrain.columns[np.where(feature_importances >0)[0]]
    
        rf = RandomForestRegressor(n_estimators = 100).fit(xTrain[nonzero_features],yTrain)
        test_pred = rf.predict(xTest[nonzero_features])
        train_pred = rf.predict(xTrain[nonzero_features])
    
        train_error =  np.sqrt(np.mean((yTrain -train_pred)**2))
        test_error = np.sqrt(np.mean((yTest -test_pred)**2))

        return([feature_importances,test_error,len(nonzero_features),train_error])
    
    if problem_type == 'Classification':
        
        loss = -cp.sum(cp.multiply(yTrain, tree_pred@ w) - cp.logistic(tree_pred @ w))
        objective =  (1/len(yTrain))*loss + lambd*cp.norm(cp.matmul(indicators,w),1)
        prob = cp.Problem(cp.Minimize(objective) )
        prob.solve(solver = cp.MOSEK ,mosek_params = {mosek.dparam.optimizer_max_time: 100.0})
        weights = np.asarray(w.value)
        low_values_flags = np.abs(weights) < threshold  # Where values are low
        weights[low_values_flags] = 0
        tree_ind = np.where(weights >0)[0]

        if len(tree_ind)==0:
            train_error =  np.sqrt(np.mean((yTrain )**2))
            test_error = np.sqrt(np.mean((yTest )**2))
            return([[],test_error,0,train_error])

        importances = np.array([np.array(row[2]) for row in tree_results])
        feature_importances = np.mean(importances[tree_ind],axis = 0)
        nonzero_features = xTrain.columns[np.where(feature_importances >0)[0]]
    
        rf = RandomForestClassifier(n_estimators = 100).fit(xTrain[nonzero_features],yTrain)
        
        test_pred = rf.predict_proba(xTest[nonzero_features])[:,1]
        train_pred = rf.predict_proba(xTrain[nonzero_features])[:,1]

        train_error =  0#np.mean(yTrain != train_pred)
        test_error = sklearn.metrics.roc_auc_score(yTest,test_pred)

        return([feature_importances,test_error,len(nonzero_features),train_error])
    
def sparse_tree_ensemble(arg):
    build_tree_method = arg[-1]
    tree_results = build_tree_method(arg)
    return solve_step_nonsketch(arg,tree_results)


### Experiment

In [ ]:
def run_experiment(arg,ntrials,nfeatures,threshold):
    pool = Pool(mp.cpu_count())
    test_error_result = []
    nonzero_result = []
    train_error_result = []
    count_array = []
  
    #bisection function
    LL = 0
    RL = 1
    to_find = 0
    counter = 0
    while to_find <= nfeatures:
        try:
            counter = counter + 1 
            trial = 0
            result1= []
            arg_list = []
            lambd = (LL + RL)/2
            while trial < ntrials:
                arg[5] = lambd
                arg_list.append(arg)
                result = sparse_tree_ensemble(arg)
                result1.append(result)
                trial = trial + 1
            #result1 = pool.map(sparse_tree_ensemble, arg_list)
            test_acc = [row[1] for row in result1]
            nonzero = [row[2] for row in result1]

            train_acc = [row[3] for row in result1]

            test_error_result.append(test_acc)
            nonzero_result.append(nonzero)
            count_array = np.append(count_array,nonzero)

            train_error_result.append(train_acc)

            freq = pd.DataFrame(np.column_stack(np.unique(count_array, return_counts = True)),columns = ['value','counts'])
            print(freq.head(10))

            mode_feature_current = np.mean(nonzero)

            count_to_find = freq.loc[freq['value']==to_find]['counts'].values
            print(to_find,mode_feature_current,lambd)

            if len(count_to_find) > 0:
                if count_to_find > threshold:
                    RL = lambd
                    LL = 0
                    to_find = to_find + 1
                    counter = 0

                elif np.abs(mode_feature_current-to_find) >= 1:
                    if mode_feature_current < to_find:
                        RL = lambd 
                    else:
                        LL = lambd

            elif mode_feature_current < to_find:
                RL = lambd
            elif mode_feature_current >= to_find:
                LL = lambd
                
            elif counter >= 10:
                    RL = lambd
                    LL = 0
                    to_find = to_find + 1
                    counter = 0

            print('counter:', counter)
        except:
            print('Solver Failed')
        pool.close()
    return test_error_result,nonzero_result,train_error_result

def baseline(xTrain,yTrain,xTest,yTest,problem_type,nfeatures):
    
    if problem_type == 'Regression':
        model = RandomForestRegressor(n_estimators = 100)
    if problem_type == 'Classification':
            model = RandomForestClassifier(n_estimators = 100)
    
    ### RF feature select baseline
    rf = model.fit(xTrain,yTrain)
    imp = pd.DataFrame(np.column_stack((xTrain.columns,rf.feature_importances_)),columns = ['features','scores']).sort_values('scores',ascending = False)
    print(imp)
    acc =[]
    n_features = []
    se = []
    for i in range(1,nfeatures):
        to_use = imp.head(i)['features'].values
        trial = 0
        acc1 = []
        while trial < 10:
            rf1 = model.fit(xTrain[to_use],yTrain)
            pred = rf1.predict_proba(xTest[to_use])[:,1]
            
            if problem_type == 'Regression':
                acc1.append(np.sqrt(np.mean((yTest-pred)**2)))
                
            if problem_type == 'Classification':
                acc1.append(sklearn.metrics.roc_auc_score(yTest,pred))
                
            trial = trial+1
            
        acc.append(np.mean(acc1))
        se.append(np.std(acc1))
        n_features.append(i)
    return acc,n_features,se